In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA default;

In [0]:
%sql
DROP TABLE IF EXISTS default.courses;
DROP TABLE IF EXISTS default.courses_sink;

In [0]:
# 1) aktive Streams stoppen (falls vorhanden)
for s in spark.streams.active:
    s.stop()

# 2. Checkpoint-Verzeichnis des Streams löschen
# Dadurch "vergisst" der Stream seine komplette Historie.
checkpoint_path = "/Volumes/workspace/default/volume/_chk/courses_vw_to_sink"
dbutils.fs.rm(checkpoint_path, recurse=True)

print("Alle Tabellen und Checkpoints wurden zurückgesetzt. Sie können den Prozess neu starten.")

Alle Tabellen und Checkpoints wurden zurückgesetzt. Sie können den Prozess neu starten.


# Vorbereitung: Datei einlesen und als Tabelle abspeichern

In [0]:
%python
dataset_school = "/Volumes/workspace/default/volume"

all_files = dbutils.fs.ls(dataset_school)
json_files = [f for f in all_files if f.name.endswith(".json")]

display(json_files)

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/volume/courses.json,courses.json,329,1756365194000
dbfs:/Volumes/workspace/default/volume/enrollments.json,enrollments.json,371,1756365193000
dbfs:/Volumes/workspace/default/volume/nested_json.json,nested_json.json,1186,1756365194000
dbfs:/Volumes/workspace/default/volume/students.json,students.json,322,1756365194000
dbfs:/Volumes/workspace/default/volume/students_update.json,students_update.json,216,1756365193000


In [0]:
# JSON lesen:
courses_df = spark.read.json("/Volumes/workspace/default/volume/courses.json")
courses_df.createOrReplaceTempView("courses")

In [0]:
# Courses speichern
courses_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.courses")

# Structured Streaming

Read Stream von PySpark API

In [0]:
# spark.readStream ermöglicht das Abfragen einer Delta-Tabelle als 
# Streaming-Quelle und erstellt daraus ein Streaming-DataFrame.
stream_df = spark.readStream.table("workspace.default.courses")
stream_df.createOrReplaceTempView("courses_streaming_tmp_vw")  # das ist jetzt eine STREAMING View

Write Stream

In [0]:
# Optional: Zieltabelle mit gleichem Schema anlegen (falls noch nicht da)
spark.sql("""
  CREATE TABLE IF NOT EXISTS workspace.default.courses_sink
  LIKE workspace.default.courses
""")

# Aus der STREAMING-View lesen
df = spark.table("courses_streaming_tmp_vw")  # bleibt Streaming-DF

# Stream -> Delta-Tabelle (Serverless: availableNow + eigener Checkpoint!)
q = (
  df.writeStream
    .format("delta")
    .outputMode("append")
    .trigger(availableNow=True) # Dieser Trigger ist in der Free Edition zwingend
    .option("checkpointLocation",
            "/Volumes/workspace/default/volume/_chk/courses_vw_to_sink")
    .toTable("workspace.default.courses_sink")
)

# Bei availableNow wartet’s bis fertig und stoppt dann
q.awaitTermination()

# Ergebnis checken
display(spark.table("workspace.default.courses_sink"))

category,course_id,instructor,price,title
History,1235,Manual Insert,150,Another course
History,1234,Manual Insert,150,A new course
Technology,101,Dr. Smith,850,Data Engineering
AI,102,Dr. Lee,1200,Machine Learning
Philosophy,103,Dr. Kim,600,Ethics in AI


In [0]:
%sql
-- Daten prüfen (variante SQL)
SELECT * FROM workspace.default.courses_sink

category,course_id,instructor,price,title
History,1235,Manual Insert,150,Another course
History,1234,Manual Insert,150,A new course
Technology,101,Dr. Smith,850,Data Engineering
AI,102,Dr. Lee,1200,Machine Learning
Philosophy,103,Dr. Kim,600,Ethics in AI


Daten einfügen

In [0]:
%sql
INSERT INTO workspace.default.courses VALUES ('History', 1234, 'Manual Insert', 150, 'A new course');
INSERT INTO workspace.default.courses VALUES ('History', 1235, 'Manual Insert', 150, 'Another course');

num_affected_rows,num_inserted_rows
1,1


In [0]:
# Write Stream nochmals ausführen weil availableNow=True (limitierung Free Edition).
# DANACH hier weiter gehen

In [0]:
%sql
-- Inser ist in der Quelltabelle nicht vorhanden. Grund: Cache ist nicht aktualisiert
SELECT * FROM courses

category,course_id,instructor,price,title
Technology,101,Dr. Smith,850,Data Engineering
AI,102,Dr. Lee,1200,Machine Learning
Philosophy,103,Dr. Kim,600,Ethics in AI


In [0]:
%sql
--- die sink-Tabelle zeigt uns die vollständigen Daten (die Stream-Wahrheit)
SELECT * FROM workspace.default.courses_sink ORDER by course_id DESC

category,course_id,instructor,price,title
History,1235,Manual Insert,150,Another course
History,1234,Manual Insert,150,A new course
Philosophy,103,Dr. Kim,600,Ethics in AI
AI,102,Dr. Lee,1200,Machine Learning
Technology,101,Dr. Smith,850,Data Engineering


In [0]:
%sql
-- Cache löschen um ebenfalls die Wahrheit aus der Quelltabelle zu analysieren (variante Free Edition)
-- Die Bedingung "IS NOT NULL" für den Primary Key ändert das Ergebnis nicht,
-- aber zwingt Spark zu einer neuen Ausführung. 
-- Alternative Cluster neu starten
SELECT * FROM workspace.default.courses WHERE course_id IS NOT NULL

category,course_id,instructor,price,title
Technology,101,Dr. Smith,850,Data Engineering
AI,102,Dr. Lee,1200,Machine Learning
Philosophy,103,Dr. Kim,600,Ethics in AI
History,1235,Manual Insert,150,Another course
History,1234,Manual Insert,150,A new course


%md
## Streaming Data Manipulations in Python

In [0]:
# Ziel-Verzeichnis für den Checkpoint der Aggregation
instructor_agg_chk = "/Volumes/workspace/default/volume/_chk/courses_by_instructor_agg_final"

# 1. Alte Ergebnistabelle löschen, damit sie neu erstellt werden kann
spark.sql("DROP TABLE IF EXISTS workspace.default.courses_by_instructor")

# 2. Checkpoint löschen. KRITISCH! Sonst wird der Stream keine Daten verarbeiten.
dbutils.fs.rm(instructor_agg_chk, recurse=True)

print("✅ Altes Ergebnis und Checkpoint gelöscht. Bereit für den neuen Lauf.")

✅ Altes Ergebnis und Checkpoint gelöscht. Bereit für den neuen Lauf.


In [0]:
import pyspark.sql.functions as F

# Definiere den Stream, der von der bestehenden Quelltabelle liest
stream_df = spark.readStream.table("workspace.default.courses")

# Definiere die Aggregation
agg_df = (
    stream_df.groupBy("instructor")
             .agg(F.count("course_id").alias("total_courses"))
)

# Starte den Stream.
# Er wird die Ergebnistabelle "courses_by_instructor" selbst erstellen.
q = (
  agg_df.writeStream
    .format("delta")
    .outputMode("complete")
    .trigger(availableNow=True)
    .option("checkpointLocation", instructor_agg_chk)
    .toTable("workspace.default.courses_by_instructor")
)

# Warten, bis der Stream-Job fertig ist
q.awaitTermination()

print("✅ Streaming-Aggregation abgeschlossen und Ergebnistabelle neu erstellt.")

✅ Streaming-Aggregation abgeschlossen und Ergebnistabelle neu erstellt.


In [0]:
print("⬇️ Aktuelles Ergebnis der Aggregation:")

# Abfrage der neu erstellten Zieltabelle
display(spark.table("workspace.default.courses_by_instructor"))

⬇️ Aktuelles Ergebnis der Aggregation:


instructor,total_courses
Manual Insert,2
Dr. Smith,1
Dr. Kim,1
Dr. Lee,1


Insert

In [0]:
%sql
INSERT INTO workspace.default.courses VALUES ('History', 1236, 'Manual Insert2', 180, 'A new course');
INSERT INTO workspace.default.courses VALUES ('History', 1237, 'Manual Insert2', 180, 'Another new course');

num_affected_rows,num_inserted_rows
1,1


In [0]:
# Zelle mit Write Stream nochmals ausführen weil availableNow=True (limitierung Free Edition).
# DANACH hier weiter gehen

In [0]:
%sql
SELECT * FROM workspace.default.courses ORDER BY course_id DESC

category,course_id,instructor,price,title
History,1237,Manual Insert2,180,Another new course
History,1236,Manual Insert2,180,A new course
History,1235,Manual Insert,150,Another course
History,1234,Manual Insert,150,A new course
Philosophy,103,Dr. Kim,600,Ethics in AI
AI,102,Dr. Lee,1200,Machine Learning
Technology,101,Dr. Smith,850,Data Engineering


In [0]:
%sql
SELECT * FROM workspace.default.courses_by_instructor order by total_courses DESC

instructor,total_courses
Manual Insert2,2
Manual Insert,2
Dr. Kim,1
Dr. Smith,1
Dr. Lee,1


# Aufräumen

In [0]:
# 1) aktive Streams stoppen
for s in spark.streams.active:
    s.stop()

# 2. Checkpoint-Verzeichnis des Streams löschen
# Dadurch "vergisst" der Stream seine komplette Historie.
checkpoint_path = "/Volumes/workspace/default/volume/_chk/courses_vw_to_sink"
dbutils.fs.rm(checkpoint_path, recurse=True)

print("Alle Tabellen und Checkpoints wurden zurückgesetzt. Sie können den Prozess neu starten.")

Alle Tabellen und Checkpoints wurden zurückgesetzt. Sie können den Prozess neu starten.


In [0]:
%sql
DROP TABLE IF EXISTS courses;
DROP TABLE IF EXISTS courses_sink;
DROP TABLE IF EXISTS courses_by_instructor;
